

# AMLS Assignment
## Task A_3: CNN on BreastMNIST Dataset - Hyperparameter Selection

Explore range and combinations of Hyperparameters for CNN based model on the BreastMNIST dataset.

## Import libraries
The required libraries for this notebook include numpy, matplotlib, tensorflow, medmnist and also a custom built dataload and utility library.

In [71]:
## import libraries
## first enable autoreload during development so latest (new) version local code library is reloaded on execution 
## can be commented out when local code development completed to avoid overhead
%load_ext autoreload
%autoreload 2
    
import io
import time                          ## to enable delays between file saves
import numpy as np                   ## array manipulation
import matplotlib.pyplot as plt      ## for graphing
## import tensorflow
import tensorflow as tf              ## tensor, model functions
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPooling2D, Dropout
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.losses import BinaryCrossentropy, Hinge
## local code library, - developed for these specific tasks and includes all references to MedMNIST specific library
import AMLS_common as ac

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Set base parameters
Including hyper parameters and data set specifics

In [72]:
## set up lists and parameters
test_list     = []
run_list      = []
batch_size    = 128                 ## batch size for datasets
patience      = 3                   ## number of overfitting epochs before terminating
threshold     = 0.1                 ## overfitting threshold
## control and environment (e.g. verbose) parameters
filebase      = "metrics/"          ## the folder to store the metrics and summary files generated for each run
verbose       = 0                   ## if value equals 1 then print additional process information in steps below

In [73]:
## use these lists of values to grid test hyper parameter sensitivity
epochs_list   = [10,15,20,25]                   ## set of epochs to run for
ks_list       = [2,3]                           ## kernel size
lr_list       = [0.001,0.005]                   ## learning rates
filter_list   = [16,32,64]                      ## main filter sizes to use
ly_list       = [2,3,4]                         ## number of covolution layers
dr_list       = [0.1,0.25]                      ## selected dropout rates
loss_list     = ['binary_crossentropy']         ## loss functions to use
optimise_list = ['Adam']                        ## optimisation functions
st_list       = [1,2]                           ## strides
## now set up the required hyperparameter sets
for lr in lr_list:
    for ks in ks_list:
        for ep in epochs_list:
            for fi in filter_list:
                for ly in ly_list:
                    for dr in dr_list:
                        for st in st_list:
                            for op in optimise_list:
                                for ls in loss_list:
                                    parameter = ac.HyperParameters(learning_rate=lr, 
                                                                   kernel_size=ks, 
                                                                   num_epochs=ep, 
                                                                   num_filter=fi,
                                                                   layers=ly,
                                                                   dropout_rate=dr,
                                                                   strides=st,
                                                                   padding="same",
                                                                   optimise=op,
                                                                   loss=ls)          
                                    test_list.append([parameter])
## reshape parameters into a test grid that can be read using for loop
test_grid = [hp for sublist in test_list for hp in sublist]
print("test cases:",len(test_grid))

test cases: 576


## Load and preprocess the BreastMNIST Data
We load and split the dataset into the component datasets

In [74]:
## Loading the data file using custom MedMINST loader
data_flag  = 'breastmnist'                             ## defines which dataset to load
result_set = ac.medMNIST_load(data_flag,batch_size)    ## batch size currently hardwired

## check that the loader returned data correctly and then split out
if result_set != []:
    train_dataset = result_set[0]               ## training set
    test_dataset  = result_set[1]               ## test set
    val_dataset   = result_set[2]               ## validation set

Using downloaded and verified file: C:\Users\johnc\.medmnist\breastmnist.npz
Using downloaded and verified file: C:\Users\johnc\.medmnist\breastmnist.npz
Using downloaded and verified file: C:\Users\johnc\.medmnist\breastmnist.npz


## Fit the model

Using each Hyperparameter in turn from a superset

In [75]:
## set up the variables to keep track of the hyperparameter combinations
iterations      = len(test_grid)                                      ## number of hyperparameter sets
countie         = 0                                                   ## interim count for iterations in loop
stop_overfit_cb = ac.StopOverfittingCallback(patience, threshold)     ## initialise overfitting callback
## Create instances of the dataclass from the list
for item in test_grid:
    countie += 1
    print("Run",countie,"/",iterations,"with",item)
    ## initialise tqdm callback
    tqdm_callback = ac.TqdmEpochProgress(total_epochs=item.num_epochs)
    
    ## now set the CNN model to support Learning Rate and other Hyperparameter selection analysis
    ## added options for desired number of layers with each sub section using a similar structure
    if item.layers == 2:
        model = Sequential([
            Conv2D(item.num_filter*2, kernel_size=item.kernel_size, padding=item.padding,\
                   activation=item.default_activation, input_shape=(28, 28, 1)),  ## Input layer
            Conv2D(item.num_filter, kernel_size=item.kernel_size, padding=item.padding,\
                   activation=item.default_activation),                           ## Another convolution layer
            MaxPooling2D((2, 2),strides=item.strides),                            ## Reduce the feature maps
            Dropout(item.dropout_rate),                                           ## Drop out to reduce possible over fitting
            Conv2D(item.num_filter, kernel_size=item.kernel_size, padding=item.padding,\
                   activation=item.default_activation),                           ## Another convolution layer
            MaxPooling2D((2, 2),strides=item.strides),                            ## Reduce the feature maps
            Flatten(),                                                            ## Flatten
            Dense(item.num_filter*4,activation=item.default_activation),          ## Fully connected layer
            Dense(item.num_filter*4, activation=item.default_activation),         ## Fully connected layer
            Dropout(item.dropout_rate*2),                                         ## Drop out to reduce possible over fitting
            Dense(1, activation='sigmoid')                                        ## Output layer for binary classification  
        ])
    if item.layers == 3:
        model = Sequential([
            Conv2D(item.num_filter*2, kernel_size=item.kernel_size, padding=item.padding,\
                   activation=item.default_activation, input_shape=(28, 28, 1)),  ## Input layer
            Conv2D(item.num_filter, kernel_size=item.kernel_size, padding=item.padding,\
                   activation=item.default_activation),                           ## Another convolution layer
            MaxPooling2D((2, 2),strides=item.strides),                            ## Reduce the feature maps
            Dropout(item.dropout_rate),                                           ## Drop out to reduce possible over fitting
            Conv2D(item.num_filter, kernel_size=item.kernel_size, padding=item.padding,\
                   activation=item.default_activation),                           ## Another convolution layer
            MaxPooling2D((2, 2),strides=item.strides),                            ## Reduce the feature maps
            Conv2D(item.num_filter, kernel_size=item.kernel_size, padding=item.padding,\
                   activation=item.default_activation),                           ## Another convolution layer
            MaxPooling2D((2, 2),strides=item.strides),                            ## Reduce the feature maps
            Flatten(),                                                            ## Flatten
            Dense(item.num_filter*4, activation=item.default_activation),         ## Fully connected layer
            Dense(item.num_filter*4, activation=item.default_activation),         ## Fully connected layer
            Dropout(item.dropout_rate*2),                                         ## Drop out to reduce possible over fitting
            Dense(1, activation='sigmoid')                                        ## Output layer for binary classification  
        ])
    if item.layers == 4:
        model = Sequential([
            Conv2D(item.num_filter*2, kernel_size=item.kernel_size, padding=item.padding,\
                   activation=item.default_activation, input_shape=(28, 28, 1)),  ## Input layer
            Conv2D(item.num_filter, kernel_size=item.kernel_size, padding=item.padding,\
                   activation=item.default_activation),                           ## Another convolution layer
            MaxPooling2D((2, 2),strides=item.strides),                            ## Reduce the feature maps
            Dropout(item.dropout_rate),                                           ## Drop out to reduce possible over fitting
            Conv2D(item.num_filter, kernel_size=item.kernel_size, padding=item.padding,\
                   activation=item.default_activation),                           ## Another convolution layer
            Conv2D(item.num_filter, kernel_size=item.kernel_size, padding=item.padding,\
                   activation=item.default_activation),                           ## Another convolution layer
            MaxPooling2D((2, 2),strides=item.strides),                            ## Reduce the feature maps
            Conv2D(item.num_filter*2, kernel_size=item.kernel_size, padding=item.padding,\
                   activation=item.default_activation),                           ## Another convolution layer
            MaxPooling2D((2, 2),strides=item.strides),                            ## Reduce the feature maps
            Flatten(),                                                            ## Flatten
            Dense(item.num_filter*4, activation=item.default_activation),         ## Fully connected layer
            Dense(item.num_filter*4, activation=item.default_activation),         ## Fully connected layer
            Dropout(item.dropout_rate*2),                                         ## Drop out to reduce possible over fitting
            Dense(1, activation='sigmoid')                                        ## Output layer for binary classification  
        ])   
        
    if verbose == 1:
        print(model.summary())
        
    ## Redirect the summary output to a string so ity can be stored in file
    summary_string  = io.StringIO()
    model.summary(print_fn=lambda x: summary_string.write(x + "\n"))
    summary_content = summary_string.getvalue()
    summary_string.close()

    ## Compile the model
    model.compile(optimizer=item.optimise,                                                   
                  loss     =item.loss,
                  metrics  ='acc')

    ## Fit the model using validation set for hyperparameter selection
    history = model.fit(train_dataset,
                        validation_data = val_dataset, 
                        epochs          = item.num_epochs, 
                        batch_size      = batch_size, 
                        verbose         = 0,
                        callbacks       = [stop_overfit_cb,tqdm_callback])

    ## Save results
    run_list.append(ac.hyper_process(history,summary_content,item))

print("Hyperparameter test run complete")

Run 1 / 576 with HyperParameters(learning_rate=0.001, kernel_size=2, num_epochs=10, optimise='Adam', loss='binary_crossentropy', num_filter=16, strides=1, padding='same', dropout_rate=0.1, layers=2, default_activation='relu')


Epoch Progress: 100%|██████████| 10/10 [00:05<00:00,  1.69epoch/s, loss=0.469, acc=0.806, val_loss=0.457, val_acc=0.833]


Run 2 / 576 with HyperParameters(learning_rate=0.001, kernel_size=2, num_epochs=10, optimise='Adam', loss='binary_crossentropy', num_filter=16, strides=2, padding='same', dropout_rate=0.1, layers=2, default_activation='relu')


Epoch Progress: 100%|██████████| 10/10 [00:03<00:00,  2.88epoch/s, loss=0.536, acc=0.731, val_loss=0.539, val_acc=0.731]


Run 3 / 576 with HyperParameters(learning_rate=0.001, kernel_size=2, num_epochs=10, optimise='Adam', loss='binary_crossentropy', num_filter=16, strides=1, padding='same', dropout_rate=0.25, layers=2, default_activation='relu')


...

Overfitting detected at epoch 25: Loss Gap = 0.1477
Run 575 / 576 with HyperParameters(learning_rate=0.005, kernel_size=3, num_epochs=25, optimise='Adam', loss='binary_crossentropy', num_filter=64, strides=1, padding='same', dropout_rate=0.25, layers=4, default_activation='relu')



Epoch Progress: 100%|██████████| 25/25 [03:35<00:00,  8.60s/epoch, loss=0.387, acc=0.835, val_loss=0.408, val_acc=0.859]


Run 576 / 576 with HyperParameters(learning_rate=0.005, kernel_size=3, num_epochs=25, optimise='Adam', loss='binary_crossentropy', num_filter=64, strides=2, padding='same', dropout_rate=0.25, layers=4, default_activation='relu')


Epoch Progress: 100%|██████████| 25/25 [01:29<00:00,  3.58s/epoch, loss=0.306, acc=0.875, val_loss=0.42, val_acc=0.821] 


Overfitting detected at epoch 25: Loss Gap = 0.1143
Hyperparameter test run complete


In [77]:
## Get best hyperparameter sets and both print them out and save them to parameter files that can be fed to Tune model runs
run_df,best_run,best_run2,best_run3 = ac.analyse_run(run_list," ",filebase)
print("\nRun satisfying both smallest min_loss and largest max_acc:")
if len(best_run) > 0:
    ac.process_best_run(best_run)
print("\n")
print("\nRun with largest max_acc that is plateau or increasing:")
if len(best_run2) > 0:
    ac.process_best_run(best_run2)
print("\n")
print("\nRun with smallest min_loss that is plateau or decreasing:")
if len(best_run3) > 0:
    ac.process_best_run(best_run3)

if len(run_df)>1:
    feature_importance,coef = ac.analyse_hyperparameters(run_df)
    print("\nImpact of Hyperparameters on Accuracy (from Linear Regression):")
    print(coef)
    print("\nHyperparameter Importance for Accuracy (from Random Forest):")
    print(feature_importance)
else:
    print("\n")
    print('Suppressed feature analysis as train set too small')

No single run matches both objectives, so individually
Run with the smallest min_loss:
metrics_file                             
summary_file                             
min_loss                         0.204564
max_acc                          0.915751
last_loss                        0.204564
last_acc                         0.915751
min_val_loss                     0.348678
max_val_acc                      0.897436
last_val_loss                    0.504972
last_val_acc                     0.820513
var_loss                         0.015525
var_acc                          0.005544
learning_rate                       0.001
kernel_size                             3
num_epochs                             25
num_filter                             64
strides                                 1
padding                              same
dropout_rate                          0.1
layers                                  3
optimise                             Adam
loss                  binary_cr